In [8]:
import os
import csv
import json
import pandas as pd

In [9]:
path_to_tweets = '../hierarchical_edge_bundling/tweets_marchandisation.csv'
path_to_users = 'following/'

df = pd.read_csv( path_to_tweets, sep=';')
sorted = df.sort_values(by="average_hash")

users_parsed = []
edges = "Source;Target;TypeLien;Weight;AverageHash\n"
not_found = 0
share_weight = 0.1
follow_weight = 1

In [10]:
# Créer des liens basés sur la copublication de average hashes et le follow

# pour chaque ligne du csv
for index, row in sorted.iterrows():
    user_name = row['from_user_name']
    ahash = row['average_hash']
    # pour chaque tweet qui n'a pas encore été inspecté
    for tmp_index, tmp_row in sorted.iterrows():
        if tmp_index < index:
            continue
        # qui comprend le même average hash et dont l'auteur n'est pas le même que le tweet original
        if tmp_row['average_hash'] == row['average_hash'] and tmp_row['from_user_name'] != user_name:
            # ajouter à la base de liens un lien de partage avec l'usager·e du tweet original
            edges += f"{user_name};{tmp_row['from_user_name']};Partage;{share_weight};{ahash}\n"
    try:
        # si on a pas déjà regardé les followers de l'utilisateur
        if user_name not in users_parsed:
            # on ouvre le fichier des followers
            with open(path_to_users + user_name + '.json', 'r') as f:
                # on ne garde que les followers qui sont aussi dans le corpus
                data = [user['username'] for user in json.load(
                    f) if user['username'] in list(df['from_user_name'])]
                # on créé un lien entre chacun de ces followers et l'utilisateur courant
                for user in data:
                    edges += f"{user_name};{user};Follow;{follow_weight};\n"
            users_parsed.append(user_name)
    except FileNotFoundError:
        print(user_name + " not found!")
        not_found += 1
    except json.JSONDecodeError:
        print(user_name + " error!")
        not_found += 1


In [ ]:
# Créer des liens basés sur la copublication&follow de sha1 et le follow

sorted = df.sort_values(by="sha1")
users_parsed = []
edges = "Source;Target;TypeLien;Weight;Sha1\n"
current_sha1 = None
current_user = None
for index, row in sorted.iterrows():
    user_name = row['from_user_name']
    if current_sha1 == row['sha1'] and current_user != user_name:
        try:
            with open('following/' + user_name + '.json', 'r') as f:
                data = [user['username'] for user in json.load(f)]
                if current_user in data:
                    type_link = "Partage&Follow"
                else:
                    type_link = "Partage"
                edges += f"{user_name};{current_user};{type_link};1;{current_sha1}\n"

        except FileNotFoundError:
            print(user_name + " not found!")
            not_found += 1
        except json.JSONDecodeError:
            print(user_name + " error!")
    current_sha1 = row['sha1']
    current_user = user_name
not_found


In [11]:
# Écrire le .csv

with open(f"ahash_follow_{follow_weight}_share_{share_weight}.csv", 'w', encoding='utf-8') as f:
    f.write(edges)